In [11]:
import numpy as np
from typing import List, Tuple

In [12]:
def find_min_zeros(zero_bool_mat: np.ndarray, marked_zeros: List[Tuple[int, int]]) -> None:
    """
    Find and mark a zero in the row with the fewest zeros.

    Parameters
    ----------
    zero_bool_mat : np.ndarray
        A boolean matrix where `True` represents a zero in the original cost matrix.
    marked_zeros : List[Tuple[int, int]]
        A list to store the positions of marked zeros.

    Returns
    -------
    None
        Updates `zero_bool_mat` and `marked_zeros` in place.
    """
    row_sums = np.sum(zero_bool_mat, axis=1)
    min_row = np.argmin(np.where(row_sums > 0, row_sums, np.inf))
    zero_col = np.argmax(zero_bool_mat[min_row])
    
    marked_zeros.append((min_row, zero_col))
    zero_bool_mat[min_row, :] = False
    zero_bool_mat[:, zero_col] = False

def mark_matrix(mat: np.ndarray) -> Tuple[List[Tuple[int, int]], List[int], List[int]]:
    """
    Mark zeros in the cost matrix and determine rows and columns to cover.

    Parameters
    ----------
    mat : np.ndarray
        The cost matrix.

    Returns
    -------
    Tuple[List[Tuple[int, int]], List[int], List[int]]
        A tuple containing:
        - marked_zeros: List of positions of marked zeros.
        - marked_rows: List of row indices to cover.
        - marked_cols: List of column indices to cover.
    """
    zero_bool_mat = (mat == 0)
    marked_zeros = []

    while np.any(zero_bool_mat):
        find_min_zeros(zero_bool_mat, marked_zeros)

    marked_rows = []
    marked_cols = []
    for r, c in marked_zeros:
        marked_rows.append(r)
        marked_cols.append(c)
    
    return marked_zeros, list(set(marked_rows)), list(set(marked_cols))

def adjust_matrix(mat: np.ndarray, cover_rows: List[int], cover_cols: List[int]) -> np.ndarray:
    """
    Adjust the cost matrix by subtracting the smallest uncovered value and adding it to covered elements.

    Parameters
    ----------
    mat : np.ndarray
        The cost matrix.
    cover_rows : List[int]
        Indices of rows to cover.
    cover_cols : List[int]
        Indices of columns to cover.

    Returns
    -------
    np.ndarray
        The adjusted cost matrix.
    """
    n, m = mat.shape
    min_val = np.min([mat[i, j] for i in range(n) for j in range(m)
                      if i not in cover_rows and j not in cover_cols])
    
    for i in range(n):
        for j in range(m):
            if i not in cover_rows and j not in cover_cols:
                mat[i, j] -= min_val
            elif i in cover_rows and j in cover_cols:
                mat[i, j] += min_val

    return mat

def hungarian_algorithm(mat: np.ndarray) -> List[Tuple[int, int]]:
    """
    Solve the assignment problem using the Hungarian Algorithm.

    Parameters
    ----------
    mat : np.ndarray
        The cost matrix.

    Returns
    -------
    List[Tuple[int, int]]
        A list of optimal assignments as (row, column) pairs.
    """
    mat -= np.min(mat, axis=1)[:, np.newaxis]
    mat -= np.min(mat, axis=0)
    
    while True:
        marked_zeros, marked_rows, marked_cols = mark_matrix(mat)
        
        total_lines = len(marked_rows) + len(marked_cols)
        if total_lines >= mat.shape[0]:
            break
        
        mat = adjust_matrix(mat, marked_rows, marked_cols)
    
    return marked_zeros

def ans_calculation(mat: np.ndarray, pos: List[Tuple[int, int]]) -> Tuple[float, np.ndarray]:
    """
    Calculate the total cost and assignment matrix for the optimal solution.

    Parameters
    ----------
    mat : np.ndarray
        The original cost matrix.
    pos : List[Tuple[int, int]]
        List of optimal assignments as (row, column) pairs.

    Returns
    -------
    Tuple[float, np.ndarray]
        A tuple containing:
        - total: The total cost of the optimal assignment.
        - ans_mat: The assignment matrix with selected costs.
    """
    ans_mat = np.zeros_like(mat)
    total = 0
    for r, c in pos:
        total += mat[r, c]
        ans_mat[r, c] = mat[r, c]
    
    return total, ans_mat

def main() -> None:
    """
    Main function to demonstrate the Hungarian Algorithm on a sample cost matrix.

    Returns
    -------
    None
    """
    cost_matrix = np.array([[7, 6, 2, 9, 2],
                            [6, 5, 1, 3, 9],
                            [5, 6, 8, 9, 5],
                            [6, 8, 5, 7, 6],
                            [9, 5, 6, 4, 7]])
    
    optimal_positions = hungarian_algorithm(cost_matrix.copy())
    total_cost, assignment_matrix = ans_calculation(cost_matrix, optimal_positions)
    
    print(f"Minimum Assignment Cost: {total_cost:.0f}\nAssignment Matrix:\n{assignment_matrix}")

if __name__ == '__main__':
    main()


Minimum Assignment Cost: 13
Assignment Matrix:
[[0 0 0 0 2]
 [0 0 1 0 0]
 [5 0 0 0 0]
 [0 0 0 0 0]
 [0 5 0 0 0]]
